## Get all the articles to summarize

In [ ]:
import json
import requests

host = "https://0adf-82-50-143-221.ngrok-free.app"
articles_json = requests.get(f"{host}/articlesToSummarize").json()

## Instanciate BART

In [ ]:
from transformers import BartForConditionalGeneration, AutoTokenizer

model_ckpt = "sshleifer/distilbart-cnn-6-6"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BartForConditionalGeneration.from_pretrained(model_ckpt).cuda()

## Check Cuda comparibility

In [ ]:
import torch
from tqdm import tqdm
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Tokenize and filter by number of pages
If a paper has more than 10 pages, it is filtered out as a single summary will be too long for chat gpt input


In [ ]:
page_size = 1024

paper_rows = []
skipped_papers = []
for paper in tqdm(articles_json):
    tokens = tokenizer(paper['body'], padding='max_length', return_tensors='pt').to(device)
    n_pages = math.ceil(len(tokens["input_ids"][0])/page_size)
    paper_rows.append(
            {
            "link": paper["link"],
            "body":  paper["body"],
            "summary":  "",
            "input_ids":  tokens
            }
        )

## Summarize Docs
Split tokens in pages of 1024 tokens

A single paper (document) is divided in pages (token_splits): each page is an array of tokens.

Documents_tokenized contains the list of papers that must be summarized, and it is an array of documents.

In [ ]:
max_size = 1024
for row in tqdm(paper_rows):
    n_splits = math.ceil(len(row["input_ids"][0])/max_size)
    document_text_summary = ""
    token_count = 0
    for index in list(range(n_splits)):
        if(index != n_splits-1):
            # print(str(index*max_size) + " - " + str((index+1)*max_size))
            token_splits = { "input_ids": torch.tensor(row['input_ids']['input_ids'][0][index*max_size:(index+1)*max_size]).unsqueeze(0).to(device),
                                "attention_mask": torch.tensor(row['input_ids']['attention_mask'][0][index*max_size:(index+1)*max_size]).unsqueeze(0).to(device)}
        else:
            # print(str(index*max_size) + " - " + str(len(row['input_ids']['input_ids'][0])%max_size + index*max_size))
            token_splits = { "input_ids": torch.tensor(row['input_ids']['input_ids'][0][index*max_size:len(row['input_ids']['input_ids'][0])%max_size + index*max_size]).unsqueeze(0).to(device),
                            "attention_mask": torch.tensor(row['input_ids']['attention_mask'][0][index*max_size:len(row['input_ids']['input_ids'][0])%max_size + index*max_size]).unsqueeze(0).to(device)}

        if(token_splits["input_ids"].nelement() == 0):
          continue
        doc_token_summary = model.generate(input_ids=token_splits['input_ids'],
                            attention_mask=token_splits['attention_mask'],
                            min_length=16,
                            max_length=64)
        token_count += len(doc_token_summary[0])
        extracted_summary = tokenizer.decode(doc_token_summary[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        if ("." in extracted_summary):
            document_text_summary += (".".join(extracted_summary.split(".")[0:-1])) + "\n"
        else:
            document_text_summary += extracted_summary + "\n"
 
    
    requests.post(f"{host}/updateArticle", json={
      "link": row["link"],
      "token_count": token_count,
      "summary": document_text_summary
    })
    
    # row["summary"] = document_text_summary
    # row["input_ids"] = None0

 95%|█████████▌| 1459/1532 [2:04:03<05:56,  4.88s/it] 